In [1]:
from representation_matrices_explicit import GroupD6h, GroupD6D3hC6v, BaseGroup
from graphene.graphene import GrapheneHamiltonian, CosetReps
import numpy as np
import sympy as sp

%load_ext autoreload
%autoreload 2


In [2]:
graphene_con = GrapheneHamiltonian(with_spin=False)

In [3]:
group_of_K = GroupD6D3hC6v(name='d3h', edge_x_orientation=True, is_double_group=graphene_con.with_spin)

In [ ]:
graphene_con.full_group_to_subgroup_mapping()